In [1]:
import sys
import os
import numpy as np

def tf(string): #for boolean parameter values
    if string == "False":
        return(False)
    elif string == "True":
        return(True)
    else:
        print("Bad boolean at: " + string)
        print("Defaulting to False")
        return(False)
    
def sample_dist(distname, distparams):
    #distribution types are encoded in class objects, this function
    #samples accordingly
    if distname == "fixed":
        val = distparams
    elif distname == "exponential":
        val = np.random.exponential(distparams)
    elif distname == "uniform":
        val = np.random.uniform(distparams[0], distparams[1])
    else:
        print("sample_dist (helper) error, distribution type not known.")
        print(distname)
        val = False
    return(val)

In [3]:
tf("True")
tf("False")
print(sample_dist("uniform", (0,1)))
print(sample_dist("exponential", 5))
print(sample_dist("poisson", 2))

0.752674522554
9.04835370349
sample_dist (helper) error, distribution type not known.
poisson
False


In [4]:
import sys
import os
import numpy as np


class parameters:
    def __init__(self):
        #all parameter values
        self.ALL = ["SIMULATION_TIME", "TIME_RESOLUTION", "DRIVE_TIME_DIST", "DRIVE_TIME_DIST",
                    "EXOGENOUS_INTERARRIVAL", "SERVICE_TIME_DIST", "RENEGE_TIME", "NUM_SPOTS", 
                    "ROAD_NETWORK", "VERBOSE"]
        #If param file passes single float, float is diagonalized across a network matrix
        self.diagonalize = ["EXOGENOUS_INTERARRIVAL", "SERVICE_TIME", "RENEGE_TIME", "NUM_SPOTS"]

        #Simulator parameters
        self.SIMULATION_TIME = 1000.0
        self.TIME_RESOLUTION = 0.001
        self.VERBOSE = True
        
        #Simulator statistics to collect, list of keywords
        self.STATS = ["occupancy"]

        #Network parameters
        self.EXOGENOUS_INTERARRIVAL = 1.5 
        self.SERVICE_TIME = 5.0
        self.SERVICE_TIME_DIST = "fixed" #distribution for service times, eg exponential or fixed
        self.RENEGE_TIME = 0.0
        self.NUM_SPOTS = 5
        self.DRIVE_TIME = 1.0
        self.DRIVE_TIME_DIST = "exponential" #distribution for drive times, eg exponential or fixed

        #Directed network topologies
        #Default network is 2-cycle
        self.ROAD_NETWORK = np.array([[0,1],
                                      [1,0]])
        self.networks = ["ROAD_NETWORK"] #list is hold over for multiple overlaid networks (parking garages)
        
    def read(self, inFilePath):
        #read parameter file
        inFile = open(inFilePath, 'r')
        lines = inFile.readlines()
        for line in lines:
            tokens = line.strip().split("=")
            if line[0] == "#" or line.strip() == "":
                pass
            elif tokens[1].strip()[0] == "/":
                try:
                    arr = np.loadtxt(tokens[1].strip(), dtype=np.dtype(float), delimiter = ",")
                    setattr(self, tokens[0].strip(), arr)
                except Exception as err:
                    print("Error reading parameter file path at line: ")
                    print(line)
                    print(err)
            elif tokens[1].strip()[0] == "T" or tokens[1].strip()[0] == "F":
                setattr(self, tokens[0].strip(), tf(tokens[1].strip()))
            else:
                try:
                    #float parameters
                    setattr(self, tokens[0].strip(), float(tokens[1].strip()))
                except:
                    #string parameters
                    setattr(self, tokens[0].strip(), tokens[1].strip())

        for att in self.diagonalize:
            if type(getattr(self, att)) != np.ndarray:
                setattr(self, att, getattr(self, att) * np.eye(self.ROAD_NETWORK.shape[1]))
            
    def write(self, outFilePath, ident):
        with open(outFilePath + "/" + ident + "_PARAMS.txt", 'w') as f:
            for att in self.ALL:
                if att in self.diagonalize:
                    f.write(att + " = " + outFilePath + "/" + ident + "_" + att + ".txt \n")
                    try:
                        np.savetxt(outFilePath + "/" + ident + "_" + att + ".txt", getattr(self, att), delimiter=",")
                    except:
                        out = getattr(self, att) * np.eye(self.ROAD_NETWORK.shape[1])
                        np.savetxt(outFilePath + "/" + ident + "_" + att + ".txt", out, delimiter=",")
                elif att in self.networks:
                    f.write(att + " = " + outFilePath + "/" + ident + "_" + att + ".txt \n")
                    np.savetxt(outFilePath + "/" + ident + "_" + att + ".txt", getattr(self, att), delimiter=",")
                else:
                    f.write(att + " = " + str(getattr(self, att)) + "\n")


In [5]:
params = parameters()

In [6]:
params.read('../../../test/five_block_test/test_params.txt')

In [7]:
newArrivalArray = 2.0 * np.eye(params.ROAD_NETWORK.shape[0])
setattr(params, 'EXOGENOUS_INTEARRIVAL', newArrivalArray)
newSimTime = 2000.0
setattr(params, 'SIMULATION_TIME', newSimTime)
setattr(params, 'SERVICE_TIME_DIST', 'exponential')
params.write('../../../test/five_block_test/', 'write_test') #output dir, identifier

In [10]:
class nqueue: #this queue class serves as the queue type for both streets and blockfaces
    def __init__(self, category, index, paramsInst):
        self.INDEX = index
        self.CATEGORY = category
        if category == "blockface":
            self.EXOGENOUS_INTERARRIVAL = paramsInst.EXOGENOUS_INTERARRIVAL
            self.SERVICE_TIME = paramsInst.SERVICE_TIME[index, index]
            self.SERVICE_TIME_DIST = paramsInst.SERVICE_TIME_DIST
            self.NUM_SPOTS = paramsInst.NUM_SPOTS[index, index]
            self.ACTIVE_SERVERS = np.zeros((int(self.NUM_SPOTS),))
            self.NEIGHBORING_BLOCKS = []
            
            #statistics
            self.TOTAL = 0
            self.EXOGENOUS_ARRIVALS = 0
        
            #stats requiring multiple measurements
            self.OCCUPANCY = []
            self.STATIONARY = []
            
        elif category == "street":
            self.ORIGIN = index[0]
            self.DESTINATION = index[1]
            self.SERVICE_TIME = paramsInst.DRIVE_TIME
            self.SERVICE_TIME_DIST = paramsInst.DRIVE_TIME_DIST
            self.NUM_SPOTS = np.inf #infinite server queue
            self.ACTIVE_SERVERS = np.zeros((20,)) #assuming a likely maximum capacity, 
                                     #will dynamically expand if needed
            
            #statistics
            self.TOTAL = 0
        else:
            category == "garage" #empty category for now
            print("Unprototyped nqueue category: garage")
            pass
        
    def get_service_time(self):
        service_time = sample_dist(self.SERVICE_TIME_DIST, self.SERVICE_TIME)
        return(service_time)    
    

        
        

In [11]:
a = nqueue("blockface", 1, params)
print(a.get_service_time())
print(len(a.ACTIVE_SERVERS))

3.92674092896
5


In [8]:
class blockfaceNet:
    def __init__(self, paramInstance, stats=[]):
        self.PARAMS = paramInstance
        self.TIMER = 0.0
        self.STOPWATCH = []
        self.NUM_MEASUREMENTS = 1000.0 #number of measurements for in situ sensors
        
        num_blocks = self.PARAMS.ROAD_NETWORK.shape[1]
        
        self.BLOCKFACES = {}
        for ii in range(num_blocks):
            self.BLOCKFACES[ii] = nqueue("blockface", ii, self.PARAMS)
            neighboring_blocks = [ j for j, x in enumerate(self.PARAMS.ROAD_NETWORK[ii,:]) if x == 1 ]
            self.BLOCKFACES[ii].NEIGHBORING_BLOCKS = neighboring_blocks
        
        #new arrival timer        
        arrival_times = np.diag(self.PARAMS.EXOGENOUS_INTERARRIVAL)
        self.INJECTION_BLOCKS = [ i for i, x in enumerate(arrival_times) if float(x) != 0.0 ]
        self.NEW_ARRIVAL_TIMER = np.zeros((len(self.INJECTION_BLOCKS),))
        for bi in range(len(self.INJECTION_BLOCKS)):
            curr = self.INJECTION_BLOCKS[bi]
            inittime = self.BLOCKFACES[curr].get_service_time()
            self.NEW_ARRIVAL_TIMER[bi] = inittime   
        
        self.STREETS = {} #key is origin blockface, value is dict of streets with 
                          #destinations by key
        for origin in range(num_blocks):
            self.STREETS[origin] = {}
            for dest_n in self.BLOCKFACES[origin].NEIGHBORING_BLOCKS:
                self.STREETS[origin][dest_n] = nqueue("street", (origin,dest_n), self.PARAMS)
    
    def step_time(self):
        self.TIMER += self.PARAMS.TIME_RESOLUTION
        
        #countdowns
        for block in self.BLOCKFACES.keys():
            self.BLOCKFACES[block].ACTIVE_SERVERS -= self.PARAMS.TIME_RESOLUTION
            for st in self.STREETS[block]: #streets originating from the current block
                self.STREETS[block][st].ACTIVE_SERVERS -= self.PARAMS.TIME_RESOLUTION
                
        if self.TIMER % (self.PARAMS.SIMULATION_TIME / 10) <= self.PARAMS.TIME_RESOLUTION:
            if self.PARAMS.VERBOSE == True:
                print(str(int(self.TIMER / (self.PARAMS.SIMULATION_TIME/100))) + "% complete")
        
        #sensor measurements
        if self.TIMER % (self.PARAMS.SIMULATION_TIME / self.NUM_MEASUREMENTS) <= self.PARAMS.TIME_RESOLUTION:
            self.STOPWATCH.append(self.TIMER)
            
            if "occupancy" in self.PARAMS.STATS:
                self.update_occupancy()
        
        
    #simulation statistics for blockfaces
    def update_occupancy(self):
        for block in self.BLOCKFACES.keys():
            spots = float(self.BLOCKFACES[block].NUM_SPOTS)
            parked = float(len([ i for i, x in enumerate(self.BLOCKFACES[block].ACTIVE_SERVERS) 
                                if x > self.PARAMS.TIME_RESOLUTION ]))
            self.BLOCKFACES[block].OCCUPANCY.append(parked/spots)
            
    #def update_stationary(self):

In [12]:
QNet = blockfaceNet(params)
print(QNet.NEW_ARRIVAL_TIMER)


[ 11.35710103   0.23330745  23.81838918   6.53773517   1.80650517
   6.10592176   3.33418519   3.27030928   1.83345631   1.56644498]
0% complete
10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
100% complete
[-2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996]
[-2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996
 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996
 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996
 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996 -2000.00099996]


In [ ]:
while QNet.TIMER < QNet.PARAMS.SIMULATION_TIME:
    QNet.step_time()
    
print(QNet.BLOCKFACES[1].ACTIVE_SERVERS)
print(QNet.STREETS[1][3].ACTIVE_SERVERS)

In [134]:
print(QNet.BLOCKFACES[1].OCCUPANCY)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

Specific object classes: queues, blockfaces, streets, and vehicles

In [73]:
a = nqueue("blockface", 1, params)
a.BLOCKFACE.get_service_time()

5.0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


AttributeError: nqueue instance has no attribute 'BLOCKFACE'

In [54]:
a = nqueue("street", 1, params)
print(a.get_service_time())
print(len(a.ACTIVE_SERVERS))

0.1
0


In [122]:
np.concatenate( (np.zeros((1,)),np.zeros((1,))) )

array([ 0.,  0.])